# Import Libraries

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from lazypredict.Supervised import LazyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Import Dataset

In [29]:
# fetch dataset 
raw_df = pd.read_csv('dataset/Framingham Dataset.csv')

In [30]:
raw_df

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,CVD,HYPERTEN,TIMEAP,TIMEMI,TIMEMIFC,TIMECHD,TIMESTRK,TIMECVD,TIMEDTH,TIMEHYP
0,2448,1,195.00,39,106.00,70.00,0,0.00,26.97,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
1,2448,1,209.00,52,121.00,66.00,0,0.00,NaN,0,...,1,0,8766,6438,6438,6438,8766,6438,8766,8766
2,6238,2,250.00,46,121.00,81.00,0,0.00,28.73,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
3,6238,2,260.00,52,105.00,69.50,0,0.00,29.43,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
4,6238,2,237.00,58,108.00,66.00,0,0.00,28.50,0,...,0,0,8766,8766,8766,8766,8766,8766,8766,8766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11622,9998212,1,173.00,46,126.00,82.00,0,0.00,19.17,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,0
11623,9998212,1,153.00,52,143.00,89.00,0,0.00,25.74,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,0
11624,9999312,2,196.00,39,133.00,86.00,1,30.00,20.91,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,4201
11625,9999312,2,240.00,46,138.00,79.00,1,20.00,26.39,0,...,0,1,8766,8766,8766,8766,8766,8766,8766,4201


# EDA

In [31]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11627 entries, 0 to 11626
Data columns (total 39 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   RANDID    11627 non-null  int64  
 1   SEX       11627 non-null  int64  
 2   TOTCHOL   11218 non-null  float64
 3   AGE       11627 non-null  int64  
 4   SYSBP     11627 non-null  float64
 5   DIABP     11627 non-null  float64
 6   CURSMOKE  11627 non-null  int64  
 7   CIGPDAY   11548 non-null  float64
 8   BMI       11575 non-null  float64
 9   DIABETES  11627 non-null  int64  
 10  BPMEDS    11034 non-null  float64
 11  HEARTRTE  11621 non-null  float64
 12  GLUCOSE   10187 non-null  float64
 13  educ      11332 non-null  float64
 14  PREVCHD   11627 non-null  int64  
 15  PREVAP    11627 non-null  int64  
 16  PREVMI    11627 non-null  int64  
 17  PREVSTRK  11627 non-null  int64  
 18  PREVHYP   11627 non-null  int64  
 19  TIME      11627 non-null  int64  
 20  PERIOD    11627 non-null  in

In [32]:
# Initialize empty lists for categorical, numerical, and binary features
categorical = []
numerical = []

# Iterate through columns and append to appropriate list
for column in raw_df.columns:
    if raw_df[column].dtype == 'object':
        categorical.append(column)
    elif raw_df[column].dtype in ['float64', 'int64']:
        numerical.append(column)
        
print('Categorical:', categorical)
print('Numerical:', numerical)


Categorical: []
Numerical: ['RANDID', 'SEX', 'TOTCHOL', 'AGE', 'SYSBP', 'DIABP', 'CURSMOKE', 'CIGPDAY', 'BMI', 'DIABETES', 'BPMEDS', 'HEARTRTE', 'GLUCOSE', 'educ', 'PREVCHD', 'PREVAP', 'PREVMI', 'PREVSTRK', 'PREVHYP', 'TIME', 'PERIOD', 'HDLC', 'LDLC', 'DEATH', 'ANGINA', 'HOSPMI', 'MI_FCHD', 'ANYCHD', 'STROKE', 'CVD', 'HYPERTEN', 'TIMEAP', 'TIMEMI', 'TIMEMIFC', 'TIMECHD', 'TIMESTRK', 'TIMECVD', 'TIMEDTH', 'TIMEHYP']


In [33]:
# # Calculate number of rows and columns for the grid
# num_cols = 3  # Number of columns in the grid
# num_rows_numerical = (len(numerical) + num_cols - 1) // num_cols  # Calculate number of rows needed for numerical features
# num_rows_categorical = (len(categorical) + num_cols - 1) // num_cols  # Calculate number of rows needed for categorical features

# # Plot numerical features in a grid
# plt.figure(figsize=(20, 4 * num_rows_numerical))  # Adjust height based on number of rows

# for i, feature in enumerate(numerical):
#     plt.subplot(num_rows_numerical, num_cols, i + 1)
#     sns.histplot(data=raw_df, x=feature, bins=15, hue='CVD')
#     plt.title(f'Histogram of {feature}')
#     plt.xlabel(feature)
#     plt.ylabel('Frequency')

# plt.tight_layout()
# plt.show()


# # Plot categorical features in a grid
# plt.figure(figsize=(20, 4 * num_rows_categorical))  # Adjust height based on number of rows

# for i, feature in enumerate(categorical):
#     plt.subplot(num_rows_categorical, num_cols, i + 1)
#     sns.countplot(y=feature, data=raw_df, hue='CVD')
#     plt.title(f'Bar Plot of {feature}')
#     plt.xlabel('Count')
#     plt.ylabel(feature)

# plt.tight_layout()
# plt.show()

In [34]:
# # Get value counts for all columns
# value_counts = {column: raw_df[column].value_counts() for column in raw_df.columns}

# # Print value counts for all columns
# for column, counts in value_counts.items():
#     print(f"Value counts for column '{column}':")
#     print(counts)
#     print()

# Preprocessing
## Handling Missing Values

In [35]:
# counter percentage missing value in dataset (Function)
def missing_value(data_frame):
    total = data_frame.isnull().sum().sort_values(ascending=False)
    percentage = (data_frame.isnull().sum() / data_frame.isnull().count() * 100).sort_values(ascending=False)
    dtypes = data_frame.dtypes
    return pd.concat([total, percentage, dtypes], axis=1, keys=['Total', 'Percentage', 'Dtypes'])

In [36]:
missing_value(raw_df)

,Total,Percentage,Dtypes
LDLC,8601,73.97,float64
HDLC,8600,73.97,float64
GLUCOSE,1440,12.38,float64
BPMEDS,593,5.10,float64
TOTCHOL,409,3.52,float64
educ,295,2.54,float64
CIGPDAY,79,0.68,float64
BMI,52,0.45,float64
HEARTRTE,6,0.05,float64
TIMEAP,0,0.00,int64


In [37]:
def impute_median(data):
    return data.fillna(data.median())

raw_df['TOTCHOL'] = raw_df['TOTCHOL'].transform(impute_median)
raw_df['CIGPDAY'] = raw_df['CIGPDAY'].transform(impute_median)
raw_df['BMI'] = raw_df['BMI'].transform(impute_median)
raw_df['BPMEDS'] = raw_df['BPMEDS'].transform(impute_median)
raw_df['HEARTRTE'] = raw_df['HEARTRTE'].transform(impute_median)
raw_df['GLUCOSE'] = raw_df['GLUCOSE'].transform(impute_median)
raw_df['educ'] = raw_df['educ'].transform(impute_median)

In [38]:
missing_value(raw_df)

,Total,Percentage,Dtypes
LDLC,8601,73.97,float64
HDLC,8600,73.97,float64
CVD,0,0.00,int64
DEATH,0,0.00,int64
ANGINA,0,0.00,int64
HOSPMI,0,0.00,int64
MI_FCHD,0,0.00,int64
ANYCHD,0,0.00,int64
STROKE,0,0.00,int64
RANDID,0,0.00,int64


## Outliers

In [39]:
# # Assuming `raw_df` is your dataframe and `numerical` is a list of numerical column names
# fig, axes = plt.subplots(nrows=num_rows_numerical, ncols=num_cols, figsize=(15, 5 * num_rows_numerical))

# # Flatten the axes array to make it easier to iterate over
# axes = axes.flatten()

# # Create box plots for each selected feature
# for i, feature in enumerate(numerical):
#     sns.boxplot(raw_df[feature], color='blue', ax=axes[i])
#     axes[i].set_title(f'{feature} Distribution')
#     axes[i].set_xlabel(f'Amount Spent on {feature}')
#     axes[i].set_ylabel('Amount')

# # Remove any extra subplots if there are fewer features than subplots
# for j in range(i + 1, num_rows_numerical * num_cols):
#     fig.delaxes(axes[j])

# # Adjust layout
# plt.tight_layout()
# plt.show()


In [40]:
raw_df = raw_df[raw_df['CIGPDAY'] <= 40]
raw_df = raw_df[(raw_df['DIABP'] > 40) & (raw_df['DIABP'] < 140)]
raw_df = raw_df[(raw_df['BMI'] <= 50) & (raw_df['BMI'] >= 20)]
raw_df = raw_df[raw_df['HEARTRTE'] <= 200]
raw_df = raw_df[raw_df['GLUCOSE'] < 450]
raw_df = raw_df[raw_df['TOTCHOL'] < 500]

In [41]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import numpy as np

# # Assuming `raw_df` is your dataframe and `numerical` is a list of numerical column names
# fig, axes = plt.subplots(nrows=num_rows_numerical, ncols=num_cols, figsize=(15, 5 * num_rows_numerical))

# # Flatten the axes array to make it easier to iterate over
# axes = axes.flatten()

# # Create box plots for each selected feature
# for i, feature in enumerate(numerical):
#     sns.boxplot(raw_df[feature], color='blue', ax=axes[i])
#     axes[i].set_title(f'{feature} Distribution')
#     axes[i].set_xlabel(f'Amount Spent on {feature}')
#     axes[i].set_ylabel('Amount')

# # Remove any extra subplots if there are fewer features than subplots
# for j in range(i + 1, num_rows_numerical * num_cols):
#     fig.delaxes(axes[j])

# # Adjust layout
# plt.tight_layout()
# plt.show()


## Drop Absurd data

In [42]:
raw_df.drop(columns=['LDLC', 'HDLC'], inplace=True)

In [43]:
filtered_df = raw_df[(raw_df['CVD'] == 0) & ((raw_df['MI_FCHD'] == 1) |(raw_df['HOSPMI'] == 1)| (raw_df['ANYCHD'] == 1) | (raw_df['STROKE'] == 1) | (raw_df['ANGINA'] == 1))]

indices_to_drop = filtered_df.index
raw_df.drop(indices_to_drop, inplace=True)

In [44]:
raw_df.drop(columns=['TIMEHYP', 'TIMEDTH', 'TIMESTRK', 'TIMECHD', 'TIMEMI', 'TIME', 'TIMEAP','TIMECVD', 'TIMEMIFC'], inplace=True)

In [45]:
raw_df.drop(columns=['STROKE', 'HYPERTEN', 'ANYCHD', 'HOSPMI', 'MI_FCHD'], inplace=True)

# Features Correlations

In [46]:
# # Calculate correlations with 'Class_Mapped'
# correlations = raw_df.corr()['CVD'].drop('CVD')

# # Plotting the correlations with explicit hue assignment
# plt.figure(figsize=(10, 6))
# sns.barplot(x=correlations.index, y=correlations.values, hue=correlations.index, dodge=False, palette='viridis', legend=False)

# plt.xlabel('Features')
# plt.ylabel('Correlation with Class_Mapped')
# plt.title('Correlation between Features and Class_Mapped')
# plt.xticks(rotation=45)
# plt.grid(True, axis='y', linestyle='--', alpha=0.7)

# plt.tight_layout()
# plt.show()

In [47]:
raw_df

,RANDID,SEX,TOTCHOL,AGE,SYSBP,DIABP,CURSMOKE,CIGPDAY,BMI,DIABETES,...,educ,PREVCHD,PREVAP,PREVMI,PREVSTRK,PREVHYP,PERIOD,DEATH,ANGINA,CVD
0,2448,1,195.00,39,106.00,70.00,0,0.00,26.97,0,...,4.00,0,0,0,0,0,1,0,0,1
1,2448,1,209.00,52,121.00,66.00,0,0.00,25.48,0,...,4.00,0,0,0,0,0,3,0,0,1
2,6238,2,250.00,46,121.00,81.00,0,0.00,28.73,0,...,2.00,0,0,0,0,0,1,0,0,0
3,6238,2,260.00,52,105.00,69.50,0,0.00,29.43,0,...,2.00,0,0,0,0,0,2,0,0,0
4,6238,2,237.00,58,108.00,66.00,0,0.00,28.50,0,...,2.00,0,0,0,0,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11621,9998212,1,185.00,40,141.00,98.00,0,0.00,25.60,0,...,3.00,0,0,0,0,1,1,0,0,0
11623,9998212,1,153.00,52,143.00,89.00,0,0.00,25.74,0,...,3.00,0,0,0,0,1,3,0,0,0
11624,9999312,2,196.00,39,133.00,86.00,1,30.00,20.91,0,...,3.00,0,0,0,0,0,1,0,0,0
11625,9999312,2,240.00,46,138.00,79.00,1,20.00,26.39,0,...,3.00,0,0,0,0,0,2,0,0,0


# Split

In [48]:
def split_data(X, y, test_size, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

# Imbalance

In [49]:
from imblearn.over_sampling import SMOTE

def oversample(df):
    X_sample = df.drop(columns=['CVD'])
    y_sample = df['CVD']
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_sample, y_sample)
    oversampled_df = pd.DataFrame(X_resampled, columns=X_sample.columns)
    oversampled_df['CVD'] = y_resampled
    return oversampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

def undersample(df):
    minority_class = df[df['CVD'] == 1]
    majority_class = df[df['CVD'] == 0]
    
    # Determine the size of the minority class
    n_minority = len(minority_class)
    
    # Sample from the majority class to match the size of the minority class
    majority_class_undersampled = majority_class.sample(n=n_minority, random_state=42)
    
    # Concatenate minority and undersampled majority classes
    undersampled_df = pd.concat([minority_class, majority_class_undersampled])
    
    return undersampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Standardize

In [50]:
def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

# Classifier

In [51]:
def train_classifier(classifier, X_train, y_train):
    if classifier == 'dt':
        param_grid = {
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
        clf = DecisionTreeClassifier()
    elif classifier == 'rf':
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'bootstrap': [True, False]
        }
        clf = RandomForestClassifier()
    elif classifier == 'svm':
        clf = SVC()
    elif classifier == 'knn':
        param_grid = {
            'n_neighbors': [3, 5, 7, 9],
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
        }
        clf = KNeighborsClassifier()
    elif classifier == 'xgb':
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 6, 9],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0]
        }
        clf = XGBClassifier()
    else:
        raise ValueError(f"Classifier '{classifier}' not supported.")
    
    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
    
    grid_search.fit(X_train, y_train)

    return grid_search

In [52]:
def run_experiment(df, balance_method, test_size, classifier):
    if balance_method == 'oversample':
        df = oversample(df)
    elif balance_method == 'undersample':
        df = undersample(df)
    
    X = df.drop(columns=['CVD'])
    y = df['CVD']
    
    # Split the data
    X_train, X_test, y_train, y_test = split_data(X, y, test_size)
    
    # Scale the data
    X_train, X_test = scale_data(X_train, X_test)
    
     # Train classifier based on the classifier parameter
    clf = train_classifier(classifier, X_train, y_train)
    
    best_cls = clf.best_estimator_
    y_pred = best_cls.predict(X_test)
    
    # Get the classification report as a dict
    report = classification_report(y_test, y_pred, output_dict=True)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return report, conf_matrix, best_cls

In [53]:
# Define the scenarios
scenarios = [
    ('dt','imbalance', 0.1),
    # ('dt','imbalance', 0.2),
    # ('dt','imbalance', 0.3),
    # ('dt','oversample', 0.1),
    # ('dt','oversample', 0.2),
    # ('dt','oversample', 0.3),
    # ('dt','undersample', 0.1),
    # ('dt','undersample', 0.2),
    # ('dt','undersample', 0.3),
    # ('rf','imbalance', 0.1),
    # ('rf','imbalance', 0.2),
    # ('rf','imbalance', 0.3),
    # ('rf','oversample', 0.1),
    ('rf','oversample', 0.2),
    # ('rf','oversample', 0.3),
    # ('rf','undersample', 0.1),
    # ('rf','undersample', 0.2),
    # ('rf','undersample', 0.3),
    ('knn','imbalance', 0.1),
    # ('knn','imbalance', 0.2),
    # ('knn','imbalance', 0.3),
    # ('knn','oversample', 0.1),
    # ('knn','oversample', 0.2),
    # ('knn','oversample', 0.3),
    # ('knn','undersample', 0.1),
    # ('knn','undersample', 0.2),
    # ('knn','undersample', 0.3),
    # ('xgb','imbalance', 0.1),
    # ('xgb','imbalance', 0.2),
    # ('xgb','imbalance', 0.3),
    # ('xgb','oversample', 0.1),
    ('xgb','oversample', 0.2),
    # ('xgb','oversample', 0.3),
    # ('xgb','undersample', 0.1),
    # ('xgb','undersample', 0.2),
    # ('xgb','undersample', 0.3),
    # ('svm','imbalance', 0.1),
    # ('svm','imbalance', 0.2),
    # ('svm','imbalance', 0.3),
    # ('svm','oversample', 0.1),
    # ('svm','oversample', 0.2),
    # ('svm','oversample', 0.3),
    # ('svm','undersample', 0.1),
    # ('svm','undersample', 0.2),
    # ('svm','undersample', 0.3),
]

# Create an empty dataframe to store the results
results_df = pd.DataFrame()

# Loop through each scenario
for classifier, balance_method, test_size in scenarios:
    # Run the experiment
    report, conf_matrix, best_cls = run_experiment(raw_df, balance_method, test_size, classifier)
    
    # Flatten the classification report dict
    report_flat = {}

     # Add the scenario details
    report_flat['scenario'] = f'{classifier}_{balance_method}_split_{int(test_size*100)}%'
    report_flat['best_hyperparameter'] = f'{best_cls}'

    for key_outer, value_outer in report.items():
        if isinstance(value_outer, dict):
            for key_inner, value_inner in value_outer.items():
                report_flat[f'{key_outer}-{key_inner}'] = value_inner
        else:
            report_flat[key_outer] = value_outer
    
   
    
    # Append the result to the dataframe
    results_df = results_df.append(report_flat, ignore_index=True)


    # Print and plot the confusion matrix
    # print(f"\nConfusion Matrix for {balance_method} with test size {test_size*100}%:")
    # print(conf_matrix)
    # plt.figure(figsize=(8, 6))
    # sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
    #             xticklabels=['No CVD', 'CVD'], yticklabels=['No CVD', 'CVD'])
    # plt.xlabel('Predicted')
    # plt.ylabel('Actual')
    # plt.title(f'Confusion Matrix: {balance_method} - Test Size {int(test_size*100)}%')
    # plt.show()

desired_columns = [
    'scenario', 'best_hyperparameter', 'accuracy',
    '0-precision', '0-recall', '0-f1-score', '0-support',
    '1-precision', '1-recall', '1-f1-score', '1-support',
    'macro avg-precision', 'macro avg-recall', 'macro avg-f1-score', 'macro avg-support',
    'weighted avg-precision', 'weighted avg-recall', 'weighted avg-f1-score', 'weighted avg-support'
]

# Reorder columns in results_df
results_df = results_df[desired_columns]

results_df


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 243 candidates, totalling 1215 fits


,scenario,best_hyperparameter,accuracy,0-precision,0-recall,0-f1-score,0-support,1-precision,1-recall,1-f1-score,1-support,macro avg-precision,macro avg-recall,macro avg-f1-score,macro avg-support,weighted avg-precision,weighted avg-recall,weighted avg-f1-score,weighted avg-support
0,dt_imbalance_split_10%,"DecisionTreeClassifier(max_depth=10, min_sampl...",0.87,0.87,0.96,0.91,745,0.85,0.60,0.71,267,0.86,0.78,0.81,1012,0.87,0.87,0.86,1012
1,rf_oversample_split_20%,"RandomForestClassifier(bootstrap=False, n_esti...",0.89,0.88,0.90,0.89,1474,0.90,0.88,0.89,1472,0.89,0.89,0.89,2946,0.89,0.89,0.89,2946
2,knn_imbalance_split_10%,"KNeighborsClassifier(n_neighbors=9, weights='d...",0.85,0.87,0.95,0.90,745,0.80,0.59,0.68,267,0.83,0.77,0.79,1012,0.85,0.85,0.84,1012
3,xgb_oversample_split_20%,"XGBClassifier(base_score=None, booster=None, c...",0.91,0.89,0.93,0.91,1474,0.93,0.89,0.91,1472,0.91,0.91,0.91,2946,0.91,0.91,0.91,2946
